In [ ]:
# 08_power_curves.ipynb — load, summarize, and plot power/FDR across n and rho.
from pathlib import Path
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt

pd.set_option("display.max_rows", 20)
pd.set_option("display.max_columns", 50)


In [ ]:
def find_tables_dir():
    candidates = [Path("../results/tables"), Path("results/tables"), Path("../../results/tables")]
    for d in candidates:
        if d.exists():
            return d
    # create the default under parent repo root
    root = Path(".."); (root / "results" / "tables").mkdir(parents=True, exist_ok=True)
    return root / "results" / "tables"

TABLES = find_tables_dir()
files = sorted(TABLES.glob("power_*_p*.csv"))

print("TABLES =", TABLES.resolve())
print("Found", len(files), "power_* CSVs")
if files: print("e.g.", files[0].name)

# Load if present; keep df empty if none (so rest of cells won't crash)
df = pd.concat([pd.read_csv(f).assign(file=f.name) for f in files], ignore_index=True) if files else pd.DataFrame()
df.head()

In [ ]:
def melt_prefix(df, prefix):
    cols = [c for c in df.columns if isinstance(c, str) and c.startswith(prefix)]
    if not cols: 
        return pd.DataFrame()
    id_vars = [c for c in df.columns if c not in cols]
    long = df.melt(id_vars=id_vars, value_vars=cols, var_name="metric", value_name="val")
    return long

def parse_alpha(metric):
    m = re.search(r'_(0\.05|0\.10)', str(metric))
    return m.group(1) if m else None

def parse_B(metric):
    m = re.search(r'_B(\d+)', str(metric))
    return int(m.group(1)) if m else None

def label_method(metric):
    s = str(metric)
    if "_bh_"   in s: return "BH"
    if "_by_"   in s: return "BY"
    if "_lctb_" in s: return "LCT-B"
    if "_lct_"  in s: return "LCT-N"
    return "?"

def choose_lctb_rows(long_lctb, B_choice=None):
    """
    Pick which LCT-B (B) to visualize:
      - B_choice is an int: keep that B only (if present)
      - B_choice is None: choose the largest B within each (file, model, p, n, rho, alpha) group
    """
    if long_lctb.empty:
        return long_lctb
    if B_choice is not None:
        return long_lctb[long_lctb["B"] == int(B_choice)].copy()
    idx = (long_lctb
           .groupby(["file","model","p","n","rho","alpha"])["B"]
           .idxmax())
    return long_lctb.loc[idx].copy()

In [ ]:
if df.empty:
    print("No power_* CSVs loaded — run scripts/run_power_curves.py first.")
else:
    # Power
    pwr_parts = []
    for pref in ["power_bh_", "power_by_", "power_lct_", "power_lctb_"]:
        L = melt_prefix(df, pref)
        if L.empty: 
            continue
        L["alpha"]  = L["metric"].apply(parse_alpha).fillna("-")
        L["B"]      = L["metric"].apply(parse_B)
        L["method"] = L["metric"].apply(label_method)
        pwr_parts.append(L.rename(columns={"val":"power"}))
    power_long = pd.concat(pwr_parts, ignore_index=True) if pwr_parts else pd.DataFrame()

    # If multiple B for LCT-B, choose one to display (largest by default)
    lctb = power_long[power_long["method"]=="LCT-B"]
    keep_lctb = choose_lctb_rows(lctb, B_choice=None)
    power_disp = pd.concat([
        power_long[power_long["method"]!="LCT-B"], 
        keep_lctb
    ], ignore_index=True)

    # FDR
    fdr_parts = []
    for pref in ["fdr_bh_", "fdr_by_", "fdr_lct_", "fdr_lctb_"]:
        L = melt_prefix(df, pref)
        if L.empty: 
            continue
        L["alpha"]  = L["metric"].apply(parse_alpha).fillna("-")
        L["B"]      = L["metric"].apply(parse_B)
        L["method"] = L["metric"].apply(label_method)
        fdr_parts.append(L.rename(columns={"val":"fdr"}))
    fdr_long = pd.concat(fdr_parts, ignore_index=True) if fdr_parts else pd.DataFrame()
    lctb_f = fdr_long[fdr_long["method"]=="LCT-B"]
    keep_lctb_f = choose_lctb_rows(lctb_f, B_choice=None)
    fdr_disp = pd.concat([
        fdr_long[fdr_long["method"]!="LCT-B"],
        keep_lctb_f
    ], ignore_index=True)

    # Aggregate
    def agg_mean_se(fr, ycol):
        if fr.empty: 
            return pd.DataFrame()
        return (fr.groupby(["file","model","p","n","rho","alpha","method"], as_index=False)[ycol]
                  .agg(mean="mean", se=lambda x: x.std(ddof=1)/np.sqrt(len(x))))
    agg_power = agg_mean_se(power_disp, "power")
    agg_fdr   = agg_mean_se(fdr_disp,   "fdr")

    display(agg_power.head())
    display(agg_fdr.head())

In [ ]:
def nice_title(file):
    m = re.search(r'power_(\w+)_p(\d+)_n(\d+)_rho([0-9.]+)_b(\d+)_R(\d+)\.csv', file)
    # files contain one (n,rho); we’ll build titles ad-hoc below
    return file

def plot_power_vs_rho(model, p, n, alpha="0.05"):
    sub = agg_power[(agg_power["model"]==model)&(agg_power["p"]==p)&(agg_power["n"]==n)&(agg_power["alpha"]==alpha)]
    if sub.empty:
        print(f"No data for {model}, p={p}, n={n}, alpha={alpha}"); 
        return
    plt.figure(figsize=(6.5,3.0))
    for m in ["BH","BY","LCT-N","LCT-B"]:
        s = sub[sub["method"]==m].sort_values("rho")
        if s.empty: continue
        plt.errorbar(s["rho"], s["mean"], yerr=s["se"], marker="o", label=m)
    plt.ylim(0,1); plt.xlabel("rho"); plt.ylabel("power")
    plt.title(f"{model} | p={p} | n={n} — Power vs rho @ α={alpha}")
    plt.legend(); plt.tight_layout(); plt.show()

def plot_power_vs_n(model, p, rho, alpha="0.05"):
    sub = agg_power[(agg_power["model"]==model)&(agg_power["p"]==p)&(agg_power["rho"]==rho)&(agg_power["alpha"]==alpha)]
    if sub.empty:
        print(f"No data for {model}, p={p}, rho={rho}, alpha={alpha}"); 
        return
    plt.figure(figsize=(6.5,3.0))
    for m in ["BH","BY","LCT-N","LCT-B"]:
        s = sub[sub["method"]==m].sort_values("n")
        if s.empty: continue
        plt.errorbar(s["n"], s["mean"], yerr=s["se"], marker="o", label=m)
    plt.ylim(0,1); plt.xlabel("n"); plt.ylabel("power")
    plt.title(f"{model} | p={p} | rho={rho:.2f} — Power vs n @ α={alpha}")
    plt.legend(); plt.tight_layout(); plt.show()

def plot_fdr_vs_rho(model, p, n, alpha="0.05"):
    sub = agg_fdr[(agg_fdr["model"]==model)&(agg_fdr["p"]==p)&(agg_fdr["n"]==n)&(agg_fdr["alpha"]==alpha)]
    if sub.empty: 
        print(f"No FDR data for {model}, p={p}, n={n}, alpha={alpha}")
        return
    plt.figure(figsize=(6.5,3.0))
    for m in ["BH","BY","LCT-N","LCT-B"]:
        s = sub[sub["method"]==m].sort_values("rho")
        if s.empty: continue
        plt.errorbar(s["rho"], s["mean"], yerr=s["se"], marker="o", label=m)
    plt.xlabel("rho"); plt.ylabel("FDR"); plt.ylim(0,1)
    plt.title(f"{model} | p={p} | n={n} — FDR vs rho @ α={alpha}")
    plt.legend(); plt.tight_layout(); plt.show()

In [ ]:
# Examples — change these as needed
for model in ["gaussian","t","laplace","exp"]:
    plot_power_vs_rho(model, p=250, n=80, alpha="0.05")
    plot_power_vs_n(model,   p=250, rho=0.30, alpha="0.05")

# Optional: FDR sanity plots
for model in ["gaussian","t","laplace","exp"]:
    plot_fdr_vs_rho(model, p=250, n=80, alpha="0.05")

In [ ]:
OUT = (TABLES.parent / "summary"); OUT.mkdir(parents=True, exist_ok=True)

if not agg_power.empty:
    out_p = OUT / "power_summary.csv"
    agg_power.to_csv(out_p, index=False)
    print("Wrote", out_p)

if not agg_fdr.empty:
    out_f = OUT / "fdr_summary.csv"
    agg_fdr.to_csv(out_f, index=False)
    print("Wrote", out_f)